In [ ]:
#imports (same as tuto ML)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder # is this really needed ?
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split, GridSearchCV, PredefinedSplit

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

%matplotlib inline

# Question 1: Propensity score matching

We preform a naive data analysis using plots and numbers.

In [ ]:
#import the data set
lalonde_df = pd.read_csv('lalonde.csv')
#give a first look
lalonde_df.head()

### 1. A naive analysis

It is very hard to determine what a "naive" researcher would think or try to determine from this data. However, we can easily imagine that the first thing he would do is split the salary (_['re78']_) data into 2 sets: treated and untreated.

In [ ]:
#masks to be used later
treated = lalonde_df.treat == 1
untreated = lalonde_df.treat == 0

#apply masks to get treated and untreated
treated_salary = lalonde_df[treated]['re78']
untreated_salary = lalonde_df[untreated]['re78']

**i - Visualizing the data:**

We suppose that the first step of the analysis would be to plot the data to be able to have a visual summary of what it says.

_**Note:** we used the [following StackOverflow code](https://stackoverflow.com/questions/8822370/plot-line-graph-from-histogram-data-in-matplotlib) to compute the histograms as a continuous function._


**leos' comment: I think it might be better to show the discrete histogram, as we don't have that many data points? why use a continuous one, if we want to use it we should justify it better**
**we could also say that this is a log-normal distribution (lot's of 0's and very few high values)**

In [ ]:
import pylab as p

val_range = (0, np.maximum(treated_salary.max(), untreated_salary.max()))

u_y, u_binEdges = np.histogram(untreated_salary, bins = 100, range=val_range)
u_centers = 0.5*(u_binEdges[1:] + u_binEdges[:-1])
p.plot(u_centers, u_y, '-')

t_y, t_binEdges = np.histogram(treated_salary, bins = 100, range=val_range)
t_centers = 0.5*(t_binEdges[1:] + t_binEdges[:-1])
p.plot(t_centers, t_y, '-')

p.show()

##### First insights:

By looking at the graph, we see a very similar distribution for both functions, except for the outliers. However, we suppose that the researcher would not care about the outliers as much as he would about the shape of the functions. They are not even visible in the plot he would draw.

Another very visible element is the fact that the function of the treated group's salaries is shifted to the bottom. Very quickly (cf paragraph below), a simple explanation arises: the number of members in the treated group is lower.

**ii - Describing the numbers**

Thus, we determined that he would only look at the basic descriptions of the data (mean, std and 5 number summary). Even though there is a "count" column in the method we use, we supposed that such a reasearcher would not get in insights from the difference between the 2 groups but solely on the description information.

In [ ]:
lalonde_df.groupby('treat')['re78'].describe()

From the numbers above, we assume that the research would extract the following information from the data :

- The untreated group has more people.
- The untreated group's salaries are homogeneous compared to the treated group's salaries (there is a smaller deviation). Moreover, the untreated group's salaries are higher (higher mean).
- However, the max salary in the treated group is 3x higher! The 1st quartile is also twice higher on the treated group.
- Finally, we have that the second and third quartiles are higher in the first group. Quartiles are more resistent to outliers, so we should put more consideration
- The interquartile distance is larger in the untreated set, we should revisit the part about homegenety based on teh standard deviation **leo's comment**

###### Second insights:

By looking at these numbers the researcher would assume that the treatment only helps the poorer and richer people while being a burden for the "middle class". 
<- why? we can't draw this conclusion like that **leo's comment**

Moreover, it has a negative impact according to the basic statistics (the group's incomes are lower in average and allocated less efficiently according to the standard deviation).

**iii - Looking at the numbers:**

As a naive researcher often bases his insights on visual representations, we assume he would want to look at the boxplts corresponding to each group to be able to compare them more easily.

In [ ]:
plt.boxplot([treated_salary, untreated_salary], labels=['treated', 'untreated'])
plt.show()

##### Third insights :

By discarding the outliers, the researcher is clearly comforted in his past insights, allowing him to formulate a conclusion.

**Conclusion**:

By merging all of the insights the researcher has drawn from the 3 steps of his analysis, he can conclude that **the treatment is ineffective**. Even though salary distributions are similar in both cases, the treated group has in average a lower salary (and only a handful of rich people get lucky). This is shown by the boxplot: the whiskers extend higher in the untreated group.

### 2. A closer look at the data

After performing a **very simplistic** analysis of the data, we can dive deeper and start looking at the whole table, namely at the other features which surely have an impact on the variable we want to understand at the end of this exercise: _['re78']_.

**i - Interval data :**

Once again, we start our analysis by visualizing the data to see if we can recognize special distributions. However, we notice simply by looking at the "head" of our table that some data is categorical and some data constitutes intervals. In the beginning, we will focus on the latter.

_**Note:** for a concise visualization, we will only include the box plots and histograms of each variable without the previously used 5 number summary._

In [ ]:
#defining list of non binary variables
intervals = ['age', 'educ', 're74', 're75', 're78']

for col in intervals:
    print('Visualization of :', col)
    plt.title("Boxplot of " + col)
    plt.boxplot([lalonde_df[untreated][col], lalonde_df[treated][col]], labels=['untreated', 'treated'])
    plt.figure()
    
    bins = np.linspace(np.min(lalonde_df[col]), np.max(lalonde_df[col]), 30)
    plt.title("Histogram of " + col)
    plt.hist(lalonde_df[untreated][col], bins, alpha=0.5)
    plt.hist(lalonde_df[treated][col], bins, alpha=0.5)
    plt.show()

From the graphs above, we can see very clearly 2 elements. First, the data distribution is always similar between the 2 groups (even though the number of participants is different due to the difference in the number of people in both groups). On top of this, 3 types of distributions pop out:
- Poisson: this is the distribution representing the **age** of participants <- rly??? that's not what I'm seeing
- Gaussian : this distribution models the **level of education** of participants
- Power law : this type of distribution is appropriate to understand the **salaries** of participants (_['re74'], ['re75'] and ['re78']_ all have the same form of distribution)

##### a. Salaries:

The main thing we need to look at is the salaries. To have a better visualization, we need to plot them using a log-log plot.

In [ ]:
#As this does not work yet, we "comment" it
"""salaries = ['re74', 're75', 're78']

from scipy.stats import itemfreq

for col in salaries:
    print('Visualization of :', col)
    plt.title("Loglog of " + col)
    plt.hist(np.log(lalonde_df[untreated][col]))
    plt.plot(np.log(lalonde_df[treated][col]))
    plt.show()"""

##### b. Evolution:

Even though it is useful to plot the salaries to see the difference between the years, it is much more useful to understand how the salary of each participant has changed over the years. To do so, we will visualize our data using a parallel plot. 

In [ ]:
#Implement parallel plot
from pandas.plotting import parallel_coordinates
parallel_coordinates(lalonde_df[untreated][['id','re74', 're75', 're78']], 'id', color='Blue', alpha=0.5)
parplot = parallel_coordinates(lalonde_df[treated][['id','re74', 're75', 're78']], 'id', color='Orange' , alpha=0.7)
#remove legend for readability
parplot.legend_.remove()
plt.title('Salary over time for each participant')
plt.xlabel('Year')
plt.ylabel('Annualy Salary')

We see that:
- the treated group started out with a lower salary
- 75 was a bad year for everybody, treated or untreated.
- the outliers are people partialy people who were already well payed in 74, partialy people who 'made it'.
- there is a lot of movement up for the treated group between 75 and 78

**Conclusion:**

By looking at the interval data, and more specificaly at the salaries of participants, we can say that …

**ii - Categorical data :**

Regarding categorical data, we should look at rates (makes much more sense than looking at just the numbers). Thus we define the rates for race, degree and mariage depending on each treatment to be able to compare them.

In [ ]:
tot_u, tot_t = lalonde_df.groupby('treat')['id'].count()

##### a. Race ratios:

We will start with race. As we do not have numbers for "White" participants, we get the number of "Blacks" and "Hispanics" for each treatment group and substract the total. We then compare the rates of each race using pie charts.

In [ ]:
blacks_u, blacks_t = lalonde_df[lalonde_df.black == 1].groupby('treat')['id'].count()
hispan_u, hispan_t = lalonde_df[lalonde_df.hispan == 1].groupby('treat')['id'].count()
whites_u, whites_t = (tot_u - blacks_u - hispan_u, tot_t - blacks_t - hispan_t)

In [ ]:
u_race_rates = ([blacks_u, hispan_u, whites_u]/tot_u)*100
t_race_rates = ([blacks_t, hispan_t, whites_t]/tot_t)*100
race_labels = 'Blacks', 'Hispanics', 'Whites'

In [ ]:
fig, ax = plt.subplots()
ax.pie(u_race_rates, labels = race_labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.title("Rates of races in the untreated group")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.pie(t_race_rates, labels = race_labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.title("Rates of races in the treated group")
plt.show()

##### b. Degree ratios:

To have a better understanding of the difference of salaries, we also need to look at the level of education of the participants of each treatment.

In [ ]:
degree_u, degree_t = lalonde_df[lalonde_df.nodegree == 0].groupby('treat')['id'].count()
no_degree_u, no_degree_t = (tot_u - degree_u, tot_t - degree_t)

In [ ]:
u_degree_rates = ([degree_u, no_degree_u]/tot_u)*100
t_degree_rates = ([degree_t, no_degree_t]/tot_t)*100
degree_labels = 'Degree', 'No degree'

In [ ]:
fig, ax = plt.subplots()
ax.pie(u_degree_rates, labels = degree_labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.title("Rate of people with degrees in the untreated group")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.pie(t_degree_rates, labels = degree_labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.title("Rate of people with degrees in the treated group")
plt.show()

##### c. Mariage ratios:

Finally, we look at the rates of married people among both groups as it is our last feature. 

In [ ]:
married_u, married_t = lalonde_df[lalonde_df.married == 1].groupby('treat')['id'].count()
not_married_u, not_married_t = (tot_u - married_u, tot_t - married_t)

In [ ]:
u_married_rates = ([married_u, not_married_u]/tot_u)*100
t_married_rates = ([married_t, not_married_t]/tot_t)*100
mariage_labels = 'Married', 'Not married'

In [ ]:
fig, ax = plt.subplots()
ax.pie(u_married_rates, labels = mariage_labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.title("Rate of people with degrees in the untreated group")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.pie(t_married_rates, labels = mariage_labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
plt.title("Rate of people with degrees in the treated group")
plt.show()

**Conclusion:**

By looking at the categorical data, we can say that …

**iii - Correlated data :**

After working on each value alone, we want to understand how each value is (linearly) linked to others by computing the Pearson correlation on each pair of features and displaying it on a heat map.

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = lalonde_df.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

### 3. A propsensity score model

In [ ]:
prop_table = lalonde_df.copy() #otherwise we modify lalonde_df when we modyfy prop_table

In [ ]:
del prop_table['re78'], prop_table['re75'] # isn't 75 already treated? or not idk

In [ ]:
X = prop_table.iloc[:, 2:] #rows age to 're74'
y = prop_table.iloc[:, 1:2] #treated or not
y = np.ravel(y) #flatten array
print('Our Y : ', y[0:5],'\nOur X', X[0:5])

In [ ]:
logistic = LogisticRegression()
logistic.fit(X, y)
print('Accuracy of prediction: ',logistic.score(X, y))

In [ ]:
print("prediction : ", logistic.predict(prop_table.iloc[:6, 2:]), ' reality :', y[0:6])
print('prediction in percent : \n', logistic.predict_proba(prop_table.iloc[:6, 2:]))

In [ ]:
propensity_scores = pd.DataFrame(logistic.predict_proba(X)) #get propensity scores for everybody
propensity_scores['y'] = y
propensity_scores['id'] = lalonde_df['id']#keepy y values for each

In [ ]:
propensity_scores.head()

The logistic.predict's first element gives us the probablility of the subject with a given background not being treated, the second element gives the probabability of them being in the treatment group.

We now use this to find a matching

### 4. Balancing the dataset via matching

Matching the two is an equivalent problem to find a matching in a bipartite graph

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite

In [ ]:
B = nx.Graph()
#creat graph with nodes as id
B.add_nodes_from(propensity_scores['id'])

In [ ]:
# add edges from each treated to each untreated subject
# with weight on each node being the difference between the two

for row_i in propensity_scores[propensity_scores['y'] ==0 ].iterrows():
    for row_j in propensity_scores[propensity_scores['y'] ==1 ].iterrows():
        B.add_edge(row_i[1]['id'],row_j[1]['id'], weight= -np.abs(row_i[1][0] - row_j[1][0]))
        #1-x to transform minimisation problem into maximisation problem

In [ ]:
matching_dict = nx.max_weight_matching(B, maxcardinality=True)

In [ ]:
#print(np.mean(treated_subjects[0]), '\n', np.mean(untreated_subjects[0])) #the difference between the two means is quite large :(
#check sum should be minimized
remaning_subjects = lalonde_df[lalonde_df['id'].isin(matching_dict)]
len(remaning_subjects)

In [ ]:
treated_subjects = remaning_subjects[remaning_subjects['treat'] == 1]
untreated_subjects = remaning_subjects[remaning_subjects['treat'] == 0]

In [ ]:
plt.boxplot([treated_subjects.re78, untreated_subjects.re78], labels=['treated', 'untreated'])
plt.show()

### 5. Balancing the groups further


### 6. A less naive analysis

# Question 2: Applied ML

First, we need to compute the TF-IDF features of our dataset, using a vectorizer. As we understood the question, what was asked was not to use any of the given datasets from sklearn, but to use all of the data. Thus, we do not use the train and test subsets given to us in sklearn, but will create our own such subsets, adding a validation subset.

Also note that we remove the headers, footers and quotes, as proposed in the <a href="http://scikit-learn.org/stable/datasets/twenty_newsgroups.html">sklearn tutorial</a> of the dataset, as to have something more realistic and without any of the metadata. Note also that we did not use the *sklearn.datasets.fetch_20newsgroups_vectorized* function that returns the TF-IDF features directly, as it would defeat the purpose of the exercise.

In [ ]:
# create the TF-IDF vectorizer
tfidf = TfidfVectorizer()

In [ ]:
# Import the data we need to use the vectorizer on. Remove metadata as proposed by sci-kit tutorial
newsgroups_all = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

As asked in the question, before seperating in subsets, we will use the vectorizer on the complete set.

In [ ]:
# vectors is a sparse matrix
vectors = tfidf.fit_transform(newsgroups_all.data)

Now we need to seperate the dataset into three sets: train, test and validation.

In [ ]:
# first we seperate train from the rest. Random_state given to have a seed.
newsgroups_train, newsgroups_inter, vect_train, vect_inter = \
    train_test_split(newsgroups_all.target, vectors, test_size=0.2, random_state=1)

# then we seperate again to get validation and test seperately
newsgroups_test, newsgroups_valid, vect_test, vect_valid = \
    train_test_split(newsgroups_inter, vect_inter, test_size=0.5, random_state=1)

## 2.

Train a random forest on your training set. Try to fine-tune the parameters of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, display a confusion matrix of your classification pipeline. Lastly, once you assessed your model, inspect the `feature_importances_` attribute of your random forest and discuss the obtained results.


Now we need to train a random forest on our training set. For this, we will use the RandomForestClassifier, as it contains the parameters talked about in the exercise. But first, we need to ask ourselves what we want to set the parameters (*max_depth* and *n_estimators*) to.

According to the ADA course, we know that the number of trees will be in the 10's and the depth will be betweem 20 to 30. Thus for the training set, we set *n_estimators* to 10 and *max_depth* to 25.

For the predictions, we can't use the training set, as we just trained on it and thus would get very good results regardless. So prediction has to be on the validation set.

In [ ]:
# need to find estimators and depth first. We use random_state to have a seed again.
clf = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=1)
clf.fit(vect_train, newsgroups_train)
pred = clf.predict(vect_valid)
metrics.f1_score(newsgroups_valid, pred, average='macro')

As we can see, predictions aren't that great.

We try to fine tune on the validation set. Note that to do this, we usethe GridSearch implemented in sklearn. We first chose he estimators between 100 and 1000 and a depth between 20 and 30 as it is what we have seen during the lessons, but seeing as the results for the best parameters were the upper limit (30 and 1000) we decided to look if it would still be the same by taking a larger upper limit (35 and 1500).

Also, as we have already our own training, validation and test sets, we need to use *PredefinedSplit* in the GridSearch.

Please note that the fit takes a lot of time to compute, as there are a very large numbers of estimators.

In [ ]:
param_grid = { 
    'n_estimators': [100, 1500],
    'max_depth': [20, 35]
}

CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid)

In [ ]:
CV_rfc.fit(vect_train, newsgroups_train)
print(CV_rfc.best_params_)

As we can see, te best resuts are when *n_estimators* is set around X and *max_depth* is set to X. 

Now we do a confusion matrix on the test set.

Now, let us inspect the `feature_importances_` attribute of our random forest.